In [1]:
import pandas as pd
import os

from formatacao_sumulas import FootballDataProcessor
from algorithm import FootballMatchRating
import numpy as np
import warnings
warnings.filterwarnings("ignore")


id_players = pd.read_excel('id_players.xlsx', index_col=0)


data_folder = '../Data/'
file_names = ['Serie_A']


In [2]:
years = range(2013, 2023)


for year in years:
    if year == 2016:
        pass
    else:
        print(year)
        df_jogos_2014 = pd.read_json(f'../Data/Serie_A_{year}_games.json')
        df_jogos_2014_ = df_jogos_2014.T


        for index, row in df_jogos_2014_.iterrows():
            processor = FootballDataProcessor(df_jogos_2014_, index-1)
            final_df = processor.process(data_folder, file_names, year)
            final_df.rename(columns={'player_id': 'player'}, inplace=True)

            final_df['player'] = final_df['player'].astype('string')
            id_players['player'] = id_players['player'].astype('string')

            final_df_ = pd.merge(final_df, id_players, on='player', how='left')

            team_1_name = df_jogos_2014_.iloc[index-1,0]
            team_2_name = df_jogos_2014_.iloc[index-1,1]


            # Criar uma instância de FootballMatchRating
            rating_calculator = FootballMatchRating(final_df_, team_1_name, team_2_name)

            # Atualizar as ratings
            updated_match_data = rating_calculator.update_ratings()
            # Garantir que 'player' em updated_match_data está como string
            updated_match_data['player'] = updated_match_data['player'].astype('string')
            # Merge para obter apenas as colunas 'rating' e 'games_played' que precisam ser atualizadas
            updated_match_data.rename(columns={'player_name_x': 'player_name'}, inplace=True)
            updated_match_data.rename(columns={'time_jogador_x': 'time_jogador'}, inplace=True)

            updated_values = id_players[['player']].merge(
                updated_match_data[['player', 'rating', 'games_played', 'player_name', 'time_jogador']],
                on='player',
                how='left'
            )

            # Atualizar apenas as linhas corespondentes em id_players
            id_players.update(updated_values)

id_players


2013
2014
2015
2017
2018
2019
2020
2021
2022


,player,rating,age,games_played,player_name,time_jogador
0,429638,1503.819624,25,131,37Pedrinho Pedro Victor Delmino ...,Atlético Mineiro / MG
1,563757,1481.557581,25,52,46Guilherme ... Guilherme Santos da Silva,Red Bull Bragantino / SP
2,643996,1500.000000,25,50,NaN,NaN
3,816320,1500.000000,25,50,NaN,NaN
4,372444,1496.179302,25,57,21Wilson Wilson Osmar Pittoni ...,Bahia / BA
...,...,...,...,...,...,...
2881,777158,1485.328989,25,67,50JESUS,Coritiba / PR
2882,356106,1483.742866,25,64,3Rodolfo Fi ... Rodolfo Filemon de O ...,Coritiba / PR
2883,162466,1536.134584,25,61,85Roger Roger de Oliveira Be...,Atlético Mineiro / MG
2884,541294,1396.425727,25,96,22Gabriel Gabriel Cordeiro Pirani,Cuiabá / MT


In [3]:
x = id_players.sort_values(by='rating', ascending=False)
x.to_excel('teste.xlsx')
x

,player,rating,age,games_played,player_name,time_jogador
2861,346636,1707.877826,25,255,6Rene Rene Rodrigues Martins,Internacional / RS
1702,633571,1693.292871,25,157,15GUSTAVO GOMEZ Gustavo Raul Gomez P ...,Palmeiras / SP
58,169050,1691.180481,25,328,21Weverton Weverton Pereira da Silva,Palmeiras / SP
1852,389378,1683.161534,25,112,19Breno Breno Henrique Vasco ...,Palmeiras / SP
607,339258,1680.564987,25,179,13Guilherme ... Guilherme Antonio Ar ...,Atlético Mineiro / MG
...,...,...,...,...,...,...
2306,164019,1276.782651,25,151,7Joao Vitor Joao Vitor Lima Gomes,CSA / AL
1972,135288,1271.138328,25,83,8Luis Ferna ... Luis Fernando Lojudi ...,Criciúma / SC
1867,188990,1268.770066,25,111,3Alan Costa Alan Henrique Costa,CSA / AL
1727,143465,1253.006699,25,157,99Alecsandro Alecsandro Barbosa F ...,CSA / AL
